In [17]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("../data/processed/2_comments_youtube_refined.csv")

In [4]:
# Colores personalizados
colors = {
    'noticiero': 'grey',
    'pro-ruso': 'red',
    'pro-ucraniano': 'blue'
}

# 2) Detección de insultos / toxicidad


### La parte de clasificación de la muestra de 9000 registros

In [ ]:
# Rutas
xlsx_sample_path = "../data/processed/comentarios_para_etiquetar_v3.xlsx"  # ~9.000
csv_insults_path = "../data/processed/3_comments_youtube_with_insults.csv"  # +300 k
xlsx_merged_out = "../data/processed/comentarios_9000_con_insultos.xlsx"

In [22]:
# 1) Cargar
df9k = pd.read_excel(xlsx_sample_path)
df_ins = pd.read_csv(csv_insults_path)

# 2) Normalizar tipos/IDs
df9k["comment_id"] = df9k["comment_id"].astype(str).str.strip()
df_ins["comment_id"] = df_ins["comment_id"].astype(str).str.strip()

In [23]:
# 4) Merge LEFT para conservar **exactamente** los 9.000
cols_keep_from_ins = ["comment_id", "insulto", "n_insultos"]
df9k_merged = df9k.merge(df_ins[cols_keep_from_ins], on="comment_id", how="left")

In [24]:
# 5) Validaciones
print("Filas muestra original:", len(df9k))
print("Filas tras merge:", len(df9k_merged))
print("Insultos NaN tras merge:", df9k_merged["insulto"].isna().sum())

Filas muestra original: 9000
Filas tras merge: 9000
Insultos NaN tras merge: 11


In [25]:
# 6) Guardar
df9k_merged.to_excel(xlsx_merged_out, index=False)
print("✅ Merge guardado en:", xlsx_merged_out)

✅ Merge guardado en: ../data/processed/comentarios_9000_con_insultos.xlsx


##### 2) Clasificación híbrida (Reglas + ML) usando insulto real

In [3]:
input_path = "../data/processed/comentarios_9000_con_insultos.xlsx"
out_path   = "../data/processed/comentarios_clasificados_9000_hibrido.xlsx"

In [4]:
df = pd.read_excel(input_path)

In [5]:
# Normalizaciones mínimas
df["comment"] = df["comment"].astype(str)
df["condiciones_cuenta"] = df["condiciones_cuenta"].astype(str).str.strip().str.lower()

In [6]:
# Si tu etiqueta humana está como "ruso/ucraniano/neutro", dejamos ese vocabulario
valid_labels = {"ruso","ucraniano","neutro"}
if "label_comentario" in df.columns:
    df["label_comentario"] = df["label_comentario"].astype(str).str.strip().str.lower()


In [7]:
# Conversión de insulto a bool (si viniese como string)
df["insulto"] = df["insulto"].map({True: True, False: False, "True": True, "False": False}).fillna(False)


C:\Users\User\AppData\Local\Temp\ipykernel_25348\3638922102.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["insulto"] = df["insulto"].map({True: True, False: False, "True": True, "False": False}).fillna(False)


In [8]:
# Limpieza del texto y creación de contexto de canal
def clean_text(s: str) -> str:
    s = re.sub(r"http\S+", " ", s)
    s = re.sub(r"@[A-Za-z0-9_]+", " ", s)
    s = re.sub(r"[^A-Za-zÀ-ÿ0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

df["comment_clean"] = df["comment"].apply(clean_text)
df["text_with_ctx"] = (("[canal:"+df["condiciones_cuenta"]+"] ").fillna("") + df["comment_clean"]).str.strip()

In [9]:
# ------------ Reglas de alta confianza ------------

praise_channel = [
    "gran canal","buen canal","buen vídeo","gran vídeo","gracias por informar","excelente análisis", 
    "buen canal","buen análisis","buen analisis","me gusta el canal","me encanta", "canal objetivo", 
    "muy claro","te felicito","gracias por el análisis", "buen resumen" 
]
russian_power = [
    "sarmat","misil","misiles","hipersónico","hipersonico","alcance nuclear",
    "no se metan con rusia","no se metan con los rusos","poder ruso","armamento ruso","potencia rusa",
    "khinzal","kinzhal","avangard","poseidon","tsirkon","zircón","isdm","kalibr"
]
peace_humanitarian = [
    "paz","alto el fuego","alto al fuego","basta de guerra","tragedia para ambos",
    "muertes","sufrimiento","fin de la guerra","paren la guerra","que pare la guerra",
    "humanitario","ayuda humanitaria","no más guerra", "no mas guerra","acuerdo de paz"
]
attack_occident = [
    "otan","cnn","occidente","eeuu","estados unidos","propaganda occidental","mentira occidental",
    "ue propaganda","nato", "occidente decadente","imperio yanqui","globalistas","agenda 2030"
]
attack_russia = [
    "putin asesino","dictador","rusia invasora","invasión rusa","invasion rusa",
    "rusos mentirosos","propaganda rusa","kremlin miente","criminal de guerra",
    "rusia criminal","imperio ruso","genocidas rusos","terrorismo ruso","criminales rusos", 
    "bot ruso"
]

In [10]:
def contains_any(s: str, bag) -> bool:
    s = s.lower()
    return any(k in s for k in bag)

def rule_classifier(row):
    text  = row["comment"].lower()
    canal = row["condiciones_cuenta"]  # 'pro-ucraniano' / 'pro-ruso' / 'neutral'
    insult = bool(row["insulto"])

    # Fuera de contexto: respetar marca si existe
    if "fuera_de_contexto" in row and str(row["fuera_de_contexto"]).lower() in {"sí","si","true"}:
        return None, "regla-fuera-contexto"

    # Insulto al presentador -> invierte bando del canal
    if insult and canal in {"pro-ucraniano","pro-ruso"}:
        inv = "ruso" if canal == "pro-ucraniano" else "ucraniano"
        return inv, "regla-insulto-inversion"

    # Elogio del canal -> sigue bando del canal
    if contains_any(text, praise_channel) and canal in {"pro-ucraniano","pro-ruso"}:
        return ("ucraniano" if canal=="pro-ucraniano" else "ruso"), "regla-elogio-canal"

    # Glorificación/validación poder militar ruso -> pro-ruso
    if contains_any(text, russian_power):
        if any(w in text for w in ["potente","poderoso","impresionante","temible","arras","reventar","alcance","amenaza","hongo","nuclear","golpear"]):
            return "ruso", "regla-poder-ruso"

    # Paz / tragedia sin culpas -> neutral
    if contains_any(text, peace_humanitarian) and not insult:
        if not contains_any(text, attack_occident) and not contains_any(text, attack_russia):
            return "neutro", "regla-paz-humanitaria"

    # Ataque a occidente en canal pro-ucraniano -> pro-ruso
    if canal == "pro-ucraniano" and contains_any(text, attack_occident):
        return "ruso", "regla-anti-occidente-en-canal-pro-ucr"

    # Ataque a Rusia/Putin en canal pro-ruso -> pro-ucraniano
    if canal == "pro-ruso" and contains_any(text, attack_russia):
        return "ucraniano", "regla-anti-rusia-en-canal-pro-ruso"

    # Negativo explícito a líderes
    if "putin" in text and any(w in text for w in ["asesino","dictador","criminal","genocida","tirano","criminal de guerra"]):
        return "ucraniano", "regla-putin-neg"
    if "zelensky" in text and any(w in text for w in ["títere","titere","payaso","actor","corrupto"]):
        return "ruso", "regla-zelensky-neg"

    # Canal neutral + elogio técnico
    if canal == "neutral" and contains_any(text, praise_channel):
        return "neutro", "regla-elogio-canal-neutral"

    return None, None

In [11]:
# Etiquetas humanas
is_humano = df["label_comentario"].isin(valid_labels)

In [12]:
# Aplicar reglas sobre NO-humanos
df["label_rule"], df["regla_aplicada"] = None, None
mask_unlabeled = ~is_humano
df.loc[mask_unlabeled, ["label_rule","regla_aplicada"]] = df.loc[mask_unlabeled].apply(
    rule_classifier, axis=1, result_type="expand"
)

In [13]:
# ------------ Entrenar ML con solo humanos ------------

df_h = df[is_humano].copy()
X = df_h["text_with_ctx"]
y = df_h["label_comentario"]

test_size = 0.2 if y.value_counts().min() >= 2 else 0.1
Xtr, Xva, ytr, yva = train_test_split(
    X, y, test_size=test_size, stratify=y if y.value_counts().min() >= 2 else None, random_state=42
)

clf = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=35000,
        ngram_range=(1,2),
        min_df=2,
        sublinear_tf=True
    )),
    ("logreg", LogisticRegression(max_iter=300, C=4.0, class_weight="balanced", n_jobs=-1))
])

clf.fit(Xtr, ytr)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=35000, min_df=2,
                                 ngram_range=(1, 2), sublinear_tf=True)),
                ('logreg',
                 LogisticRegression(C=4.0, class_weight='balanced',
                                    max_iter=300, n_jobs=-1))])

In [14]:
# Evaluación rápida opcional
if len(Xva) > 0:
    yhat = clf.predict(Xva)
    print("=== Evaluación (solo etiquetas humanas) ===")
    print(classification_report(yva, yhat, digits=3))


=== Evaluación (solo etiquetas humanas) ===
              precision    recall  f1-score   support

      neutro      0.143     0.191     0.164        47
        ruso      0.669     0.610     0.638       182
   ucraniano      0.274     0.274     0.274        62

    accuracy                          0.471       291
   macro avg      0.362     0.359     0.359       291
weighted avg      0.500     0.471     0.484       291



In [ ]:
# ==========================================
# (2) UMBRAL Y MARGEN (ABSTENCIÓN ML)
# ==========================================

In [15]:
UMBRAL_PROBA = 0.55      # confianza mínima para aceptar la predicción ML
UMBRAL_MARGEN = 0.15     # diferencia mínima entre top1 y top2

# Asegúrate de tener estas variables creadas antes:
# - mask_unlabeled: (~is_humano)
# - df["text_with_ctx"]: texto + token de canal
# - df["label_rule"]: label por reglas (o NaN si no hay)
classes = clf.named_steps["logreg"].classes_

In [18]:
def ml_scores(X_series):
    """Devuelve pred_label, proba_max y margen(top1-top2) para cada texto de X_series."""
    P = clf.predict_proba(X_series)
    top1_idx = P.argmax(axis=1)
    top1 = classes[top1_idx]
    top1_p = P[np.arange(P.shape[0]), top1_idx]
    P_sorted = -np.sort(-P, axis=1)     # ordena cada fila desc
    margen = P_sorted[:, 0] - P_sorted[:, 1]
    return top1, top1_p, margen

to_pred_mask = mask_unlabeled & df["label_rule"].isna()
if to_pred_mask.sum() > 0:
    y_ml, p_ml, m_ml = ml_scores(df.loc[to_pred_mask, "text_with_ctx"])
    df.loc[to_pred_mask, "label_ml"] = y_ml
    df.loc[to_pred_mask, "ml_proba_max"] = p_ml
    df.loc[to_pred_mask, "ml_margen"] = m_ml
else:
    df["label_ml"] = None
    df["ml_proba_max"] = np.nan
    df["ml_margen"] = np.nan

In [19]:
# ==========================================
# (3) DECISIÓN FINAL (con abstención)
# ==========================================
valid_labels = {"ruso","ucraniano","neutro"}

In [20]:
def decide(row):
    # 1) Humano manda
    lab_h = row["label_comentario"] if row["label_comentario"] in valid_labels else None
    if lab_h:
        return lab_h, "humano"

    # 2) Reglas (alta precisión)
    lab_r = row["label_rule"]
    if isinstance(lab_r, str) and (lab_r in valid_labels):
        return lab_r, "regla"

    # 3) ML con umbral y margen
    lab_m = row.get("label_ml", None)
    pmax  = row.get("ml_proba_max", np.nan)
    margen = row.get("ml_margen", np.nan)
    if (lab_m in valid_labels) and (pmax >= UMBRAL_PROBA) and (margen >= UMBRAL_MARGEN):
        return lab_m, "automatica-ml"

    # 4) Sin evidencia suficiente -> sin clasificar
    return "", "sin-clasificar"

df["label_final"], df["clasificacion_origen"] = zip(*df.apply(decide, axis=1))

In [21]:
# ==========================================
# (4) RESUMEN Y GUARDADO
# ==========================================
print("\n== Origen de clasificación ==")
print(df["clasificacion_origen"].value_counts(dropna=False))
print("\n== Distribución label_final (incluye vacíos) ==")
print(df["label_final"].value_counts(dropna=False))


== Origen de clasificación ==
clasificacion_origen
sin-clasificar    3967
automatica-ml     3580
humano            1453
Name: count, dtype: int64

== Distribución label_final (incluye vacíos) ==
label_final
             3967
ruso         3585
ucraniano     962
neutro        486
Name: count, dtype: int64


------------ Clasificación base terminada --------------

In [22]:
df.to_excel(out_path, index=False)
print("\n✅ Guardado:", out_path)


✅ Guardado: ../data/processed/comentarios_clasificados_9000_hibrido.xlsx


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   comment_id            9000 non-null   object 
 1   comment               9000 non-null   object 
 2   comment_text_length   9000 non-null   int64  
 3   user_id               9000 non-null   object 
 4   user_name             8993 non-null   object 
 5   comment_time          9000 non-null   object 
 6   comment_likes         9000 non-null   int64  
 7   total_reply_count     9000 non-null   int64  
 8   is_top_level_comment  9000 non-null   bool   
 9   video_title           9000 non-null   object 
 10  channel_title         9000 non-null   object 
 11  video_published_at    9000 non-null   object 
 12  video_views           9000 non-null   int64  
 13  video_likes           9000 non-null   int64  
 14  video_duration        9000 non-null   int64  
 15  video_tags           

-----------------------